In [22]:
df = spark.read.csv("gs://bigdataplatform-order/market-history", header=True, inferSchema=True)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39187)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39187)

In [6]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import json
import pyspark.sql.functions as F
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.functions import from_unixtime, unix_timestamp
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

In [2]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- type_id: integer (nullable = true)
 |-- average: double (nullable = true)
 |-- lowest: double (nullable = true)
 |-- highest: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- order_count: integer (nullable = true)



In [3]:
df.show()

+----------+---------+-------+-------+------+-------+--------+-----------+
|      date|region_id|type_id|average|lowest|highest|  volume|order_count|
+----------+---------+-------+-------+------+-------+--------+-----------+
|2020-01-17| 10000001|     19| 100.99|100.99| 100.99|      90|          1|
|2020-01-17| 10000001|     20| 261.97|261.97| 261.97|    8856|          3|
|2020-01-17| 10000001|     34|    3.5|   3.5|    3.5|13524267|         37|
|2020-01-17| 10000001|     35|   2.99|  2.99|   2.99|60551454|         28|
|2020-01-17| 10000001|     36|  16.99| 16.99|  16.99|  654278|         30|
|2020-01-17| 10000001|     37|    9.0|   9.0|    9.0|  664291|         24|
|2020-01-17| 10000001|     38| 161.05|161.05| 161.05|  242667|         18|
|2020-01-17| 10000001|     39|  790.0| 790.0|  790.0|  206998|         16|
|2020-01-17| 10000001|     40|  590.0| 590.0|  590.0|  388348|         14|
|2020-01-17| 10000001|    179|   4.99|  4.99|   4.99|   20000|          1|
|2020-01-17| 10000001|   

In [5]:
def datashape(data):
    print("Data shape (rows, columns):", data.count(), "x", len(data.columns))
    
datashape(df)

Data shape (rows, columns): 113185824 x 8


In [7]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [8]:
summary = df.describe().toPandas()
summary = summary.T
summary.columns = summary.iloc[0]
summary = summary.drop(summary.index[0])
summary

summary,count,mean,stddev,min,max
date,113185824,None,None,2016-08-01,2022-05-21
region_id,113185824,1.0004142767380128E7,63970.751640847964,10000001,11000031
type_id,113185824,18766.700713103437,14314.794143615642,18,64190
average,113185824,2.7671863366771884E7,3.393177227216778E8,0.01,7.0E11
lowest,113185824,2.6923971296597917E7,3.373594573818294E8,0.0,7.0E11
highest,113185824,2.8310287015332602E7,3.4195989168462205E8,0.01,7.0E11
volume,113185824,484952.38085024676,5.340557368631074E7,1,37399429275
order_count,113185824,13.025892509295156,57.67100394720171,1,8246


In [9]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

date,region_id,type_id,average,lowest,highest,volume,order_count
0,0,0,0,0,0,0,0


In [10]:
df.groupby("date").count().sort("date", ascending=True).limit(10)

date,count
2016-08-01,52520
2016-08-02,52543
2016-08-03,52045
2016-08-04,52160
2016-08-05,52400
2016-08-06,55489
2016-08-07,57108
2016-08-08,52847
2016-08-09,51617
2016-08-10,51407


In [ ]:
df.groupBy("region_id").count().sort("count", ascending=False).limit(5)

region_id,count
10000002,16811843
10000043,10352293
10000032,7738019
10000030,5737327
10000042,5599783


In [1]:
pdf = df.limit().toPandas()

NameError: name 'df' is not defined

In [13]:
#Register the dataframe as a temporary table in the SQLContext

df.registerTempTable("df")

In [14]:
#Distinct regions 

spark.sql("SELECT COUNT(DISTINCT region_id) from df")

count(DISTINCT region_id)
96


In [21]:
from pyspark.sql.functions import month
df_month = df.withColumn('month',month(df.date))
df_month.printSchema()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39187)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39187)